In [1]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from scipy import stats
import commands

In [3]:
# dir_path = "/Users/mriworkshop/Documents/TDCS/code/tdcs_thesis/"
dir_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/"
img_path = dir_path+"data/processed/NIFTI/"

# Prediction vs Theory

In [ ]:
xminusy = "36minus34" 
xminusy = "58minus56" # run 6
tdir = sub_path+"NIFTI/bmaps_numpy/"+xminusy

# Non erode
img_meas = nib.load(cwd+"/4d_fmap_Mean.nii.gz")
img_theo = nib.load(cwd+"/Bvals.nii")

data_meas = img_meas.get_data()
data_theo = img_theo.get_data()
x = data_theo.flatten()
y = data_meas.flatten()

m, b, r, p, st_er = stats.linregress(x,y) 

yfit = [b + m * xi for xi in x]
yisx = [0 + 1 * xi for xi in x]
plt.plot(x, yfit)
plt.plot(x, yisx)

plt.scatter(x, y,  color='black')
plt.axis([0,100, 0, 100])
plt.xlabel("Theory (nT)")
plt.ylabel("Physics (nT)")
plt.title("Physics vs Theory", fontsize=15)
print("r: {:.5f}, st_er: {:.6f}".format(r, st_er))
print("y = "+str(round(m,4))+"*x + "+str(round(b,4)))